In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 31024033
paper_name = 'hernandez_golshani_2019' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/table_s1.xlsx', sheet_name='Table 1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 68 x 13


In [7]:
original_data.head()

,Unnamed: 0,Unnamed: 1,ORF,Gene,ave (control),sd,n,ave (MnCl2),sd.1,n.1,Score (Control-MnCl2),Sensitivity Intensity (%),Unnamed: 12
0,NaN,1,YGL168W,HUR1,1.170865,0.036075,3,0.002035,0.002035,3,1.168830,117,NaN
1,NaN,2,YGR081C,SLX9,1.152335,0.011215,3,0.197255,0.063595,3,0.955080,98,NaN
2,NaN,3,YDR305C,HNT2,1.083995,0.091093,4,0.336840,0.185545,4,0.747155,77,NaN
3,NaN,4,YOR069W,VPS5,1.208313,0.174882,4,0.492777,0.173317,4,0.715535,80,NaN
4,NaN,5,YPR119W,CLB2,1.097477,0.052092,3,0.437713,0.091313,3,0.659764,70,NaN


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, ORF, Gene, ave (control), sd, n, ave (MnCl2), sd.1, n.1, Score (Control-MnCl2), Sensitivity Intensity (%), Unnamed: 12, orf]
Index: []


In [12]:
original_data['data'] = -pd.to_numeric(original_data['Score (Control-MnCl2)'])

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(68, 1)

# Prepare the final dataset

In [31]:
data = original_data.copy()

In [32]:
dataset_ids = [21903]
datasets = datasets.reindex(index=dataset_ids)

In [33]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [34]:
data.head()

dataset_id,21903
data_type,value
orf,
YAL020C,-0.182485
YAR002C-A,-0.308618
YBL031W,-0.314177
YBL039C,-0.278087
YBL085W,-0.162690


## Subset to the genes currently in SGD

In [35]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [36]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21903
,data_type,value
gene_id,orf,
18,YAL020C,-0.182485
1859,YAR002C-A,-0.308618
119,YBL031W,-0.314177
127,YBL039C,-0.278087
173,YBL085W,-0.162690


# Normalize

In [37]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [38]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [39]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            21903          
data_type             value    valuez
gene_id orf                          
18      YAL020C   -0.182485 -3.792697
1859    YAR002C-A -0.308618 -6.414195
119     YBL031W   -0.314177 -6.529742
127     YBL039C   -0.278087 -5.779661
173     YBL085W   -0.162690 -3.381295

# Print out

In [40]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [41]:
from IO.save_data_to_db3 import *

In [42]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 31024033...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  7.12it/s]

Updating the data_modified_on field...
